# ハイブリッドモデル（LSTM + 拡散モデル）の学習と可視化

このノートブックでは、時系列データを低周波成分（LSTM）と高周波成分（拡散モデル）に分解して学習するハイブリッドモデルの学習過程を可視化します。

In [ ]:
# CLAUDE_ADDED
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import sys
from tqdm import tqdm
import json
from datetime import datetime

# パスを追加
current_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.dirname(current_dir)
root_dir = os.path.dirname(src_dir)
sys.path.insert(0, root_dir)
sys.path.insert(0, src_dir)

from src.ImprovedHybridTransformer import HybridTrajectoryModel
from src.DataPreprocess.DataPreprocessForOverFitting import load_processed_data

## 1. データの読み込みと前処理

In [ ]:
# CLAUDE_ADDED
# データの読み込み
data_path = '../../data/Datasets/overfitting_dataset.npz'
trajectories, conditions = load_processed_data(data_path)

print(f"軌道データ形状: {trajectories.shape}")
print(f"条件データ形状: {conditions.shape}")
print(f"軌道データ範囲: [{trajectories.min():.3f}, {trajectories.max():.3f}]")
print(f"条件データ範囲: [{conditions.min():.3f}, {conditions.max():.3f}]")

In [ ]:
# CLAUDE_ADDED
# データの可視化
plt.figure(figsize=(15, 5))

# 軌道データのサンプルを表示
plt.subplot(1, 3, 1)
for i in range(min(5, len(trajectories))):
    plt.plot(trajectories[i, :, 0], trajectories[i, :, 1], alpha=0.7, label=f'Sample {i+1}')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('軌道データサンプル')
plt.legend()
plt.grid(True)

# 条件データのヒストグラム
condition_names = ['動作時間', '終点誤差', 'ジャーク']
for i in range(3):
    plt.subplot(1, 3, i+2)
    plt.hist(conditions[:, i], bins=20, alpha=0.7)
    plt.xlabel(condition_names[i])
    plt.ylabel('頻度')
    plt.title(f'{condition_names[i]}の分布')
    plt.grid(True)

plt.tight_layout()
plt.show()

## 2. データ分解の可視化

In [ ]:
# CLAUDE_ADDED
from src.ImprovedHybridTransformer.data_decomposition import DataDecomposer

# データ分解器を初期化
decomposer = DataDecomposer(window_size=10)

# サンプルデータで分解をテスト
sample_trajectories = torch.tensor(trajectories[:5], dtype=torch.float32)
low_freq, high_freq = decomposer.decompose(sample_trajectories)

# 分解結果の可視化
plt.figure(figsize=(20, 12))

for i in range(5):
    # X軸の分解
    plt.subplot(5, 6, i*6 + 1)
    plt.plot(sample_trajectories[i, :, 0], label='Original', linewidth=2)
    plt.plot(low_freq[i, :, 0], label='Low Freq', linewidth=2)
    plt.plot(high_freq[i, :, 0], label='High Freq', alpha=0.7)
    plt.title(f'Sample {i+1} - X軸分解')
    plt.legend()
    plt.grid(True)
    
    # Y軸の分解
    plt.subplot(5, 6, i*6 + 2)
    plt.plot(sample_trajectories[i, :, 1], label='Original', linewidth=2)
    plt.plot(low_freq[i, :, 1], label='Low Freq', linewidth=2)
    plt.plot(high_freq[i, :, 1], label='High Freq', alpha=0.7)
    plt.title(f'Sample {i+1} - Y軸分解')
    plt.legend()
    plt.grid(True)
    
    # 軌道平面での可視化
    plt.subplot(5, 6, i*6 + 3)
    plt.plot(sample_trajectories[i, :, 0], sample_trajectories[i, :, 1], 'k-', label='Original', linewidth=2)
    plt.plot(low_freq[i, :, 0], low_freq[i, :, 1], 'b-', label='Low Freq', linewidth=2)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Sample {i+1} - 軌道比較')
    plt.legend()
    plt.grid(True)
    
    # 高周波成分の軌道
    plt.subplot(5, 6, i*6 + 4)
    plt.plot(high_freq[i, :, 0], high_freq[i, :, 1], 'r-', alpha=0.7)
    plt.xlabel('X Residual')
    plt.ylabel('Y Residual')
    plt.title(f'Sample {i+1} - 高周波成分')
    plt.grid(True)
    
    # パワースペクトラム
    plt.subplot(5, 6, i*6 + 5)
    original_fft = np.abs(np.fft.fft(sample_trajectories[i, :, 0]))
    low_freq_fft = np.abs(np.fft.fft(low_freq[i, :, 0]))
    high_freq_fft = np.abs(np.fft.fft(high_freq[i, :, 0]))
    
    freqs = np.fft.fftfreq(len(original_fft))
    plt.semilogy(freqs[:len(freqs)//2], original_fft[:len(freqs)//2], label='Original')
    plt.semilogy(freqs[:len(freqs)//2], low_freq_fft[:len(freqs)//2], label='Low Freq')
    plt.semilogy(freqs[:len(freqs)//2], high_freq_fft[:len(freqs)//2], label='High Freq')
    plt.xlabel('Frequency')
    plt.ylabel('Power')
    plt.title(f'Sample {i+1} - パワースペクトラム')
    plt.legend()
    plt.grid(True)
    
    # 復元誤差
    plt.subplot(5, 6, i*6 + 6)
    reconstructed = decomposer.reconstruct(low_freq[i:i+1], high_freq[i:i+1])
    error = torch.abs(sample_trajectories[i] - reconstructed[0])
    plt.plot(error[:, 0], label='X Error')
    plt.plot(error[:, 1], label='Y Error')
    plt.xlabel('Time Step')
    plt.ylabel('Reconstruction Error')
    plt.title(f'Sample {i+1} - 復元誤差')
    plt.legend()
    plt.grid(True)

plt.suptitle('データ分解の詳細分析', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 3. モデルの初期化と設定

In [ ]:
# CLAUDE_ADDED
# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")

# ハイブリッドモデルの初期化
model = HybridTrajectoryModel(
    input_dim=2,  # x, y座標
    condition_dim=3,  # 動作時間、終点誤差、ジャーク
    lstm_hidden_dim=128,
    lstm_num_layers=2,
    diffusion_hidden_dim=256,
    diffusion_num_layers=4,
    moving_average_window=10,
    num_diffusion_steps=1000
).to(device)

# モデル情報を表示
model_info = model.get_model_info()
print("\nモデル情報:")
for key, value in model_info.items():
    print(f"  {key}: {value:,}" if isinstance(value, int) else f"  {key}: {value}")

In [ ]:
# CLAUDE_ADDED
# データローダーの準備
trajectories_tensor = torch.tensor(trajectories, dtype=torch.float32)
conditions_tensor = torch.tensor(conditions, dtype=torch.float32)

dataset = TensorDataset(trajectories_tensor, conditions_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

# オプティマイザーとスケジューラーの設定
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-6)

print(f"データセットサイズ: {len(dataset)}")
print(f"バッチ数: {len(train_loader)}")

## 4. 学習ループ

In [ ]:
# CLAUDE_ADDED
# 学習履歴を記録するリスト
train_losses = []
low_freq_losses = []
high_freq_losses = []
learning_rates = []

# 学習パラメータ
num_epochs = 100
log_interval = 10
save_interval = 20

# 出力ディレクトリの作成
os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/checkpoints', exist_ok=True)
os.makedirs('outputs/generated_trajectories', exist_ok=True)

print("学習開始...")
model.train()

for epoch in range(num_epochs):
    epoch_losses = []
    epoch_low_freq_losses = []
    epoch_high_freq_losses = []
    
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for batch_idx, (batch_trajectories, batch_conditions) in enumerate(progress_bar):
        batch_trajectories = batch_trajectories.to(device)
        batch_conditions = batch_conditions.to(device)
        
        optimizer.zero_grad()
        
        # 順方向計算
        outputs = model(batch_trajectories, batch_conditions)
        
        # 損失を取得
        total_loss = outputs['total_loss']
        low_freq_loss = outputs['low_freq_loss']
        high_freq_loss = outputs['high_freq_loss']
        
        # バックプロパゲーション
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # 損失を記録
        epoch_losses.append(total_loss.item())
        epoch_low_freq_losses.append(low_freq_loss.item())
        epoch_high_freq_losses.append(high_freq_loss.item())
        
        # プログレスバーを更新
        progress_bar.set_postfix({
            'Total Loss': f'{total_loss.item():.4f}',
            'Low Freq': f'{low_freq_loss.item():.4f}',
            'High Freq': f'{high_freq_loss.item():.4f}',
            'LR': f'{scheduler.get_last_lr()[0]:.2e}'
        })
    
    # エポック終了時の処理
    avg_loss = np.mean(epoch_losses)
    avg_low_freq_loss = np.mean(epoch_low_freq_losses)
    avg_high_freq_loss = np.mean(epoch_high_freq_losses)
    current_lr = scheduler.get_last_lr()[0]
    
    train_losses.append(avg_loss)
    low_freq_losses.append(avg_low_freq_loss)
    high_freq_losses.append(avg_high_freq_loss)
    learning_rates.append(current_lr)
    
    scheduler.step()
    
    # ログ出力
    if (epoch + 1) % log_interval == 0:
        print(f"\nEpoch {epoch+1}/{num_epochs}:")
        print(f"  Average Total Loss: {avg_loss:.6f}")
        print(f"  Average Low Freq Loss: {avg_low_freq_loss:.6f}")
        print(f"  Average High Freq Loss: {avg_high_freq_loss:.6f}")
        print(f"  Learning Rate: {current_lr:.2e}")
    
    # モデル保存
    if (epoch + 1) % save_interval == 0:
        checkpoint_path = f'outputs/checkpoints/hybrid_model_epoch_{epoch+1}.pth'
        model.save_model(checkpoint_path)
        print(f"  Model saved: {checkpoint_path}")

print("\n学習完了!")

## 5. 学習過程の可視化

In [ ]:
# CLAUDE_ADDED
# 学習曲線の可視化
plt.figure(figsize=(15, 10))

# 損失曲線
plt.subplot(2, 3, 1)
plt.plot(train_losses, label='Total Loss', linewidth=2)
plt.plot(low_freq_losses, label='Low Freq Loss', linewidth=2)
plt.plot(high_freq_losses, label='High Freq Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('学習損失の推移')
plt.legend()
plt.grid(True)
plt.yscale('log')

# 学習率の推移
plt.subplot(2, 3, 2)
plt.plot(learning_rates, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('学習率の推移')
plt.grid(True)
plt.yscale('log')

# 損失比率の推移
plt.subplot(2, 3, 3)
low_freq_ratio = np.array(low_freq_losses) / np.array(train_losses)
high_freq_ratio = np.array(high_freq_losses) / np.array(train_losses)
plt.plot(low_freq_ratio, label='Low Freq Ratio', linewidth=2)
plt.plot(high_freq_ratio, label='High Freq Ratio', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss Ratio')
plt.title('損失成分比率の推移')
plt.legend()
plt.grid(True)

# 損失の移動平均
plt.subplot(2, 3, 4)
window_size = 10
if len(train_losses) >= window_size:
    train_losses_smooth = np.convolve(train_losses, np.ones(window_size)/window_size, mode='valid')
    low_freq_losses_smooth = np.convolve(low_freq_losses, np.ones(window_size)/window_size, mode='valid')
    high_freq_losses_smooth = np.convolve(high_freq_losses, np.ones(window_size)/window_size, mode='valid')
    
    plt.plot(range(window_size-1, len(train_losses)), train_losses_smooth, label='Total Loss (Smooth)', linewidth=2)
    plt.plot(range(window_size-1, len(low_freq_losses)), low_freq_losses_smooth, label='Low Freq (Smooth)', linewidth=2)
    plt.plot(range(window_size-1, len(high_freq_losses)), high_freq_losses_smooth, label='High Freq (Smooth)', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss (Smoothed)')
plt.title('損失の移動平均')
plt.legend()
plt.grid(True)
plt.yscale('log')

# 最後の数エポックの詳細
plt.subplot(2, 3, 5)
last_epochs = 20
start_idx = max(0, len(train_losses) - last_epochs)
plt.plot(range(start_idx, len(train_losses)), train_losses[start_idx:], 'o-', label='Total Loss', linewidth=2)
plt.plot(range(start_idx, len(low_freq_losses)), low_freq_losses[start_idx:], 's-', label='Low Freq Loss', linewidth=2)
plt.plot(range(start_idx, len(high_freq_losses)), high_freq_losses[start_idx:], '^-', label='High Freq Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'最後の{last_epochs}エポック')
plt.legend()
plt.grid(True)

# 収束性の分析
plt.subplot(2, 3, 6)
if len(train_losses) > 1:
    loss_diff = np.diff(train_losses)
    plt.plot(loss_diff, linewidth=2, alpha=0.7)
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Epoch')
plt.ylabel('Loss Difference')
plt.title('損失の変化量')
plt.grid(True)

plt.suptitle('ハイブリッドモデル学習過程の詳細分析', fontsize=16)
plt.tight_layout()
plt.savefig('outputs/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# 学習統計の保存
training_stats = {
    'final_total_loss': float(train_losses[-1]),
    'final_low_freq_loss': float(low_freq_losses[-1]),
    'final_high_freq_loss': float(high_freq_losses[-1]),
    'min_total_loss': float(min(train_losses)),
    'epochs_trained': len(train_losses),
    'model_info': model_info
}

with open('outputs/training_stats.json', 'w') as f:
    json.dump(training_stats, f, indent=2)

print("\n最終学習統計:")
print(f"  最終総損失: {training_stats['final_total_loss']:.6f}")
print(f"  最終低周波損失: {training_stats['final_low_freq_loss']:.6f}")
print(f"  最終高周波損失: {training_stats['final_high_freq_loss']:.6f}")
print(f"  最小総損失: {training_stats['min_total_loss']:.6f}")

## 6. 生成結果の可視化

In [ ]:
# CLAUDE_ADDED
# 生成テスト
model.eval()

# テスト用の条件を準備
test_conditions = conditions_tensor[:10].to(device)  # 最初の10個の条件を使用
original_trajectories = trajectories_tensor[:10].to(device)

# 軌道生成
with torch.no_grad():
    generated_trajectories = model.generate(
        condition=test_conditions,
        sequence_length=trajectories.shape[1],
        num_samples=3  # 各条件に対して3つのサンプルを生成
    )

# CPUに移動
generated_trajectories = generated_trajectories.cpu().numpy()
test_conditions_cpu = test_conditions.cpu().numpy()
original_trajectories_cpu = original_trajectories.cpu().numpy()

print(f"生成された軌道の形状: {generated_trajectories.shape}")

In [ ]:
# CLAUDE_ADDED
# 生成結果の詳細可視化
plt.figure(figsize=(20, 15))

num_test_samples = min(5, len(test_conditions_cpu))
condition_names = ['動作時間', '終点誤差', 'ジャーク']

for i in range(num_test_samples):
    # 元の軌道と生成された軌道の比較（軌道平面）
    plt.subplot(num_test_samples, 4, i*4 + 1)
    plt.plot(original_trajectories_cpu[i, :, 0], original_trajectories_cpu[i, :, 1], 
             'k-', linewidth=3, label='Original', alpha=0.8)
    
    # 生成された3つのサンプルを表示
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_trajectories[gen_idx, :, 0], generated_trajectories[gen_idx, :, 1], 
                '--', linewidth=2, alpha=0.7, label=f'Generated {j+1}')
    
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'条件 {i+1}: 軌道比較')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    
    # X軸の時系列比較
    plt.subplot(num_test_samples, 4, i*4 + 2)
    plt.plot(original_trajectories_cpu[i, :, 0], 'k-', linewidth=3, label='Original')
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_trajectories[gen_idx, :, 0], '--', linewidth=2, alpha=0.7, label=f'Generated {j+1}')
    plt.xlabel('Time Step')
    plt.ylabel('X Position')
    plt.title(f'条件 {i+1}: X軸時系列')
    plt.legend()
    plt.grid(True)
    
    # Y軸の時系列比較
    plt.subplot(num_test_samples, 4, i*4 + 3)
    plt.plot(original_trajectories_cpu[i, :, 1], 'k-', linewidth=3, label='Original')
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_trajectories[gen_idx, :, 1], '--', linewidth=2, alpha=0.7, label=f'Generated {j+1}')
    plt.xlabel('Time Step')
    plt.ylabel('Y Position')
    plt.title(f'条件 {i+1}: Y軸時系列')
    plt.legend()
    plt.grid(True)
    
    # 条件情報の表示
    plt.subplot(num_test_samples, 4, i*4 + 4)
    condition_values = test_conditions_cpu[i]
    bars = plt.bar(condition_names, condition_values, alpha=0.7)
    plt.ylabel('値')
    plt.title(f'条件 {i+1}: パラメータ')
    plt.xticks(rotation=45)
    
    # 各バーに値を表示
    for bar, value in zip(bars, condition_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{value:.3f}', ha='center', va='bottom')
    plt.grid(True, alpha=0.3)

plt.suptitle('ハイブリッドモデル生成結果の詳細比較', fontsize=16)
plt.tight_layout()
plt.savefig('outputs/generated_trajectories/generation_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. 成分別分析

In [ ]:
# CLAUDE_ADDED
# 生成された軌道の成分分解分析
sample_generated = torch.tensor(generated_trajectories[:15], dtype=torch.float32)  # 最初の5つの条件×3サンプル
sample_original = torch.tensor(original_trajectories_cpu[:5], dtype=torch.float32)

# 元データと生成データの分解
original_low, original_high = decomposer.decompose(sample_original)
generated_low, generated_high = decomposer.decompose(sample_generated)

plt.figure(figsize=(20, 16))

for i in range(5):
    # 低周波成分の比較
    plt.subplot(5, 6, i*6 + 1)
    plt.plot(original_low[i, :, 0], original_low[i, :, 1], 'k-', linewidth=3, label='Original Low', alpha=0.8)
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_low[gen_idx, :, 0], generated_low[gen_idx, :, 1], 
                '--', linewidth=2, alpha=0.7, label=f'Generated Low {j+1}')
    plt.xlabel('X Low Freq')
    plt.ylabel('Y Low Freq')
    plt.title(f'条件 {i+1}: 低周波成分')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    
    # 高周波成分の比較
    plt.subplot(5, 6, i*6 + 2)
    plt.plot(original_high[i, :, 0], original_high[i, :, 1], 'k-', linewidth=3, label='Original High', alpha=0.8)
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_high[gen_idx, :, 0], generated_high[gen_idx, :, 1], 
                '--', linewidth=2, alpha=0.7, label=f'Generated High {j+1}')
    plt.xlabel('X High Freq')
    plt.ylabel('Y High Freq')
    plt.title(f'条件 {i+1}: 高周波成分')
    plt.legend()
    plt.grid(True)
    
    # 低周波成分のX軸時系列
    plt.subplot(5, 6, i*6 + 3)
    plt.plot(original_low[i, :, 0], 'k-', linewidth=3, label='Original')
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_low[gen_idx, :, 0], '--', linewidth=2, alpha=0.7, label=f'Gen {j+1}')
    plt.xlabel('Time Step')
    plt.ylabel('X Low Freq')
    plt.title(f'条件 {i+1}: 低周波X')
    plt.legend()
    plt.grid(True)
    
    # 高周波成分のX軸時系列
    plt.subplot(5, 6, i*6 + 4)
    plt.plot(original_high[i, :, 0], 'k-', linewidth=3, label='Original')
    for j in range(3):
        gen_idx = i * 3 + j
        plt.plot(generated_high[gen_idx, :, 0], '--', linewidth=2, alpha=0.7, label=f'Gen {j+1}')
    plt.xlabel('Time Step')
    plt.ylabel('X High Freq')
    plt.title(f'条件 {i+1}: 高周波X')
    plt.legend()
    plt.grid(True)
    
    # 低周波成分の統計比較
    plt.subplot(5, 6, i*6 + 5)
    original_low_stats = [
        torch.mean(original_low[i]).item(),
        torch.std(original_low[i]).item(),
        torch.max(original_low[i]).item() - torch.min(original_low[i]).item()
    ]
    
    generated_low_stats = []
    for j in range(3):
        gen_idx = i * 3 + j
        stats = [
            torch.mean(generated_low[gen_idx]).item(),
            torch.std(generated_low[gen_idx]).item(),
            torch.max(generated_low[gen_idx]).item() - torch.min(generated_low[gen_idx]).item()
        ]
        generated_low_stats.append(stats)
    
    x_pos = np.arange(len(original_low_stats))
    plt.bar(x_pos - 0.2, original_low_stats, 0.4, label='Original', alpha=0.8)
    
    gen_means = np.mean(generated_low_stats, axis=0)
    gen_stds = np.std(generated_low_stats, axis=0)
    plt.bar(x_pos + 0.2, gen_means, 0.4, yerr=gen_stds, label='Generated', alpha=0.8, capsize=5)
    
    plt.xticks(x_pos, ['Mean', 'Std', 'Range'])
    plt.ylabel('値')
    plt.title(f'条件 {i+1}: 低周波統計')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 高周波成分の統計比較
    plt.subplot(5, 6, i*6 + 6)
    original_high_stats = [
        torch.mean(torch.abs(original_high[i])).item(),
        torch.std(original_high[i]).item(),
        torch.max(torch.abs(original_high[i])).item()
    ]
    
    generated_high_stats = []
    for j in range(3):
        gen_idx = i * 3 + j
        stats = [
            torch.mean(torch.abs(generated_high[gen_idx])).item(),
            torch.std(generated_high[gen_idx]).item(),
            torch.max(torch.abs(generated_high[gen_idx])).item()
        ]
        generated_high_stats.append(stats)
    
    plt.bar(x_pos - 0.2, original_high_stats, 0.4, label='Original', alpha=0.8)
    
    gen_means = np.mean(generated_high_stats, axis=0)
    gen_stds = np.std(generated_high_stats, axis=0)
    plt.bar(x_pos + 0.2, gen_means, 0.4, yerr=gen_stds, label='Generated', alpha=0.8, capsize=5)
    
    plt.xticks(x_pos, ['Mean Abs', 'Std', 'Max Abs'])
    plt.ylabel('値')
    plt.title(f'条件 {i+1}: 高周波統計')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.suptitle('成分別詳細分析: 低周波（LSTM）と高周波（拡散モデル）', fontsize=16)
plt.tight_layout()
plt.savefig('outputs/generated_trajectories/component_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. 最終モデル保存と結果サマリー

In [ ]:
# CLAUDE_ADDED
# 最終モデルの保存
final_model_path = 'outputs/hybrid_model_final.pth'
model.save_model(final_model_path)
print(f"最終モデルを保存しました: {final_model_path}")

# 生成結果の保存
np.save('outputs/generated_trajectories/generated_trajectories.npy', generated_trajectories)
np.save('outputs/generated_trajectories/test_conditions.npy', test_conditions_cpu)
np.save('outputs/generated_trajectories/original_trajectories.npy', original_trajectories_cpu)

# 最終結果サマリー
final_summary = {
    'model_type': 'HybridTrajectoryModel',
    'components': {
        'low_frequency': 'LSTM',
        'high_frequency': 'Diffusion Model (MLP-based)'
    },
    'training_info': {
        'epochs_trained': len(train_losses),
        'final_total_loss': float(train_losses[-1]),
        'final_low_freq_loss': float(low_freq_losses[-1]),
        'final_high_freq_loss': float(high_freq_losses[-1]),
        'min_total_loss': float(min(train_losses))
    },
    'model_architecture': model_info,
    'data_info': {
        'input_dim': 2,
        'condition_dim': 3,
        'sequence_length': trajectories.shape[1],
        'num_samples': trajectories.shape[0]
    },
    'generation_test': {
        'test_conditions': len(test_conditions_cpu),
        'samples_per_condition': 3,
        'total_generated': generated_trajectories.shape[0]
    },
    'timestamp': datetime.now().isoformat()
}

with open('outputs/final_summary.json', 'w') as f:
    json.dump(final_summary, f, indent=2, ensure_ascii=False)

print("\n=== ハイブリッドモデル学習完了サマリー ===")
print(f"モデル種類: {final_summary['model_type']}")
print(f"構成要素:")
print(f"  - 低周波成分: {final_summary['components']['low_frequency']}")
print(f"  - 高周波成分: {final_summary['components']['high_frequency']}")
print(f"\n学習結果:")
print(f"  - 学習エポック数: {final_summary['training_info']['epochs_trained']}")
print(f"  - 最終総損失: {final_summary['training_info']['final_total_loss']:.6f}")
print(f"  - 最終低周波損失: {final_summary['training_info']['final_low_freq_loss']:.6f}")
print(f"  - 最終高周波損失: {final_summary['training_info']['final_high_freq_loss']:.6f}")
print(f"  - 最小総損失: {final_summary['training_info']['min_total_loss']:.6f}")
print(f"\nモデルパラメータ数:")
print(f"  - 総パラメータ数: {model_info['total_parameters']:,}")
print(f"  - 低周波モデル: {model_info['low_freq_parameters']:,}")
print(f"  - 高周波モデル: {model_info['high_freq_parameters']:,}")
print(f"\n生成テスト:")
print(f"  - テスト条件数: {final_summary['generation_test']['test_conditions']}")
print(f"  - 条件あたりサンプル数: {final_summary['generation_test']['samples_per_condition']}")
print(f"  - 総生成軌道数: {final_summary['generation_test']['total_generated']}")
print("\n全てのファイルがoutputsディレクトリに保存されました。")